# SQLite Excercise

### 1.  국내의 대표적인 걸그룹 또는 보이그룹 5개 이상에 대하여 다음과 같은 정보를 갖는 테이블을 만드시오. *는 Primary Key

* id(*), group_name, 구성원 수, 데뷔일자, 소속사

In [108]:
import sqlite3

In [109]:
conn = sqlite3.connect('./ex.db')

In [110]:
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS SINGERS \
                (id INT PRIMARY KEY, \
                group_name TEXT, \
                member_no INT, \
                debut TEXT, \
                company TEXT);")

In [111]:
conn.commit()

In [112]:
import pandas as pd
singers = pd.read_csv('data/singers.csv', encoding='EUC-KR')
singers

,name,member_no,debut,company
0,방탄소년단,7,2013-06-13,빅히트 엔터
1,마마무,4,2014-06-19,RBW
2,우주소녀,13,2016-02-25,스타쉽 엔터
3,블락비,7,2011-04-15,KQ 엔터
4,엑소,9,2012-04-08,SM 엔터
5,트와이스,9,2015-10-20,JYP 엔터


In [113]:
cur = conn.cursor()
sql = "INSERT INTO singers VALUES(?,?,?,?,?);"

for i in range(5):
    cur.execute(sql, (int(i + 1), 
                        singers.iloc[i,0], # name
                        int(singers.iloc[i,1]), # member_no
                        singers.iloc[i,2], # debut
                        singers.iloc[i,3]  # company
               ))
    
conn.commit()

### 2.  이들이 불렀던 노래 또는 다른 사람이 불렀던 노래 10곡 이상에 대하여 다음의 정보를 갖는 테이블을 만드시오.
* song_id(*), song_name, 그룹 id, 발표년도, 작곡가, 도입부 가사

In [114]:
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS SONGS \
                (song_id INT PRIMARY KEY, \
                song TEXT, \
                gid INT, \
                year INT, \
                composer TEXT, \
                lyrics TEXT);" )

conn.commit()

In [115]:
songs = pd.read_csv('data/songs.csv', encoding='EUC-KR')
songs

,song,gid,year,composer,lyrics
0,작은 것들을 위한 시,1,2019,Pdogg 외,모든 게 궁금해 How’s your day
1,FAKE LOVE,1,2018,Pdogg 외,널 위해서라면 난 슬퍼도 기쁜 척 할 수가 있었어
2,넌 is 뭔들,2,2016,김도훈 외,Hey 거기 미소가 예쁜 남자 바로 너
3,음오아예 (Um Oh Ah Yeh),2,2015,김도훈,oh yes 음 오 아 예 너에게 빠져들겠어 자꾸 반응하잖아
4,HER,4,2014,지코 외,Jesus 무슨 말이 필요해 모두 널 작품이라고 불러
5,으르렁,5,2013,신혁 외,나 으르렁 으르렁 으르렁 대 나 으르렁 으르렁 으르렁 대
6,MAMA,5,2012,유영진,Careless careless Shoot anonymous anonymous
7,CHEER UP,6,2016,블랙아이드필승 외,CHEER UP BABY CHEER UP BABY 좀 더 힘을 내
8,TT,6,2016,블랙아이드필승 외,이런 내 맘 모르고 너무해 너무해
9,빨간 맛,11,2017,Ludwig Lindell 외,빨간 맛 궁금해 Honey 깨물면 점점 녹아든 스트로베리 그 맛


In [116]:
cur = conn.cursor()
song_sql = 'INSERT INTO songs VALUES(?,?,?,?,?,?);'
for i in range(10):
    cur.execute(song_sql, (int(i + 1), 
                        songs.iloc[i,0], # song
                        int(songs.iloc[i,1]), # gid
                        int(songs.iloc[i,2]), # year
                        songs.iloc[i,3], # composer
                        songs.iloc[i,4]  # lyrics 
                     ))
    
conn.commit()

In [117]:
print('gid: ',songs.iloc[i,1])
print('year:', songs.iloc[i,2])

gid:  11
year: 2017


### 3. 위 두개의 테이블을 조인한 결과를 가지고 다음의 필드를 갖는 데이터 프레임을 만드시오.
* 그룹 이름, 구성원 수, 데뷔 일자, 노래 이름, 발표 년도

In [118]:
cur = conn.cursor()
join_sql = "SELECT si.group_name, si.member_no, si.debut, so.song, so.year \
                FROM singers AS si INNER JOIN songs AS so\
                ON si.id = so.gid;"
cur.execute(join_sql)
rows = cur.fetchall()
columnName = ['그룹 이름', '구성원 수', '데뷔 일자', '노래 이름', '발표 년도']
song_df = pd.DataFrame(columns=columnName)

for row in rows:
    song_df = song_df.append(pd.DataFrame([list(row)], columns=columnName), ignore_index=True)
    
song_df

,그룹 이름,구성원 수,데뷔 일자,노래 이름,발표 년도
0,방탄소년단,7,2013-06-13,작은 것들을 위한 시,2019
1,방탄소년단,7,2013-06-13,FAKE LOVE,2018
2,마마무,4,2014-06-19,넌 is 뭔들,2016
3,마마무,4,2014-06-19,음오아예 (Um Oh Ah Yeh),2015
4,블락비,7,2011-04-15,HER,2014
5,엑소,9,2012-04-08,으르렁,2013
6,엑소,9,2012-04-08,MAMA,2012


In [121]:
cur = conn.cursor()
join_sql = "SELECT si.group_name, si.member_no, si.debut, so.song, so.year \
                FROM singers AS si LEFT OUTER JOIN songs AS so\
                ON si.id = so.gid;"
cur.execute(join_sql)
rows = cur.fetchall()
columnName = ['그룹 이름', '구성원 수', '데뷔 일자', '노래 이름', '발표 년도']
song_df = pd.DataFrame(columns=columnName)

for row in rows:
    song_df = song_df.append(pd.DataFrame([list(row)], columns=columnName), ignore_index=True)
    
song_df

,그룹 이름,구성원 수,데뷔 일자,노래 이름,발표 년도
0,방탄소년단,7,2013-06-13,FAKE LOVE,2018
1,방탄소년단,7,2013-06-13,작은 것들을 위한 시,2019
2,마마무,4,2014-06-19,넌 is 뭔들,2016
3,마마무,4,2014-06-19,음오아예 (Um Oh Ah Yeh),2015
4,우주소녀,13,2016-02-25,None,None
5,블락비,7,2011-04-15,HER,2014
6,엑소,9,2012-04-08,MAMA,2012
7,엑소,9,2012-04-08,으르렁,2013


In [107]:
conn.close()

### 연습문제 3
* 사용자의 이름과, 비밀번호를 갖는 Users 테이블이 있다. 사용자의 이름과 비밀번호를 올바르게 입력하면 ‘성공’을 출력하고, 잘못 입력하면 ‘실패’를 출력하는 프로그램을 작성하시오.

In [122]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [123]:
cur.execute('CREATE TABLE IF NOT EXISTS Users \
    (uid TEXT PRIMARY KEY, \
     name TEXT NOT NULL, \
     password TEXT);')

In [124]:

users = pd.DataFrame({
    'uid': ['jhchoi','tkkim','kmsong','sylee','ewchung'],
    'name': ['최진행','김태균','송광민','이성열','정은원'],
    'password': ['1234','1234','1234','1234','1234']
})
users

,uid,name,password
0,jhchoi,최진행,1234
1,tkkim,김태균,1234
2,kmsong,송광민,1234
3,sylee,이성열,1234
4,ewchung,정은원,1234


In [125]:
cur = conn.cursor()
sql = 'INSERT INTO Users VALUES (?, ?, ?);'
for i in range(5):
    cur.execute(sql, (users.iloc[i,0],
                      users.iloc[i,1],
                      users.iloc[i,2]))
conn.commit()

In [126]:
cur = conn.cursor()
cur.execute("SELECT * FROM Users;")
rows = cur.fetchall()
for row in rows:
    print(row)

('jhchoi', '최진행', '1234')
('tkkim', '김태균', '1234')
('kmsong', '송광민', '1234')
('sylee', '이성열', '1234')
('ewchung', '정은원', '1234')


In [127]:
conn.close()

In [128]:
def checkUser(uid, password):
    print(uid, password)
    conn = sqlite3.connect('./test.db')
    cur = conn.cursor()
    sql = "SELECT name, password FROM Users WHERE uid like ?;"
    cur.execute(sql, (uid,))
    dbname, dbpassword = cur.fetchone()
    print(dbname, dbpassword)

    conn.close()

In [129]:
# 올바르게 입력한 경우
uid, password = input("uid password 입력> ").split()
checkUser(uid, password)

uid password 입력> tkkim 1234
tkkim 1234
김태균 1234


In [130]:
# UID를 잘못 입력한 경우
uid, password = input("uid password 입력> ").split()
checkUser(uid, password)

uid password 입력> tkkim 1
tkkim 1
김태균 1234


In [132]:
# UID를 잘못 입력한 경우
uid, password = input("uid password 입력> ").split()
checkUser(uid, password)

uid password 입력> tkkkim 1
tkkkim 1


TypeError: cannot unpack non-iterable NoneType object

In [133]:
CHECK_SUCCESS = 0
INVALID_UID = 1
INCORRECT_PASSWORD = 2
DATABASE_ERROR = 3

def checkUser(uid, password):
    conn = sqlite3.connect('./test.db')
    name = ''
    try:
        cur = conn.cursor()
        sql = "SELECT name, password FROM Users WHERE uid like ?;"
        cur.execute(sql, (uid,))
        dbName, dbPassword = cur.fetchone()
        if password == dbPassword:
            returnCode = CHECK_SUCCESS
            name = dbName
        else:
            returnCode = INCORRECT_PASSWORD
    except TypeError as te:
        returnCode = INVALID_UID
    except Exception as e: 
        print('예외가 발생했습니다.', e)
        returnCode = DATABASE_ERROR
    finally:
        conn.close()
        return returnCode, name

In [143]:
uid, password = input("uid password 입력> ").split()
try : 
    result, name = checkUser(uid, password)
    if result == CHECK_SUCCESS:
        print('로그인 성공, 사용자 이름 =', name)
    elif result == INVALID_UID:
        print('실패 - 잘못된 User ID')
    elif result == INCORRECT_PASSWORD:
        print('실패 - 패스워드 불일치')
    else:
        print('실패 - 데이터베이스 에러')
except:
    print('실패 - 입력값 에러')

uid password 입력> asfsaf/safas


ValueError: not enough values to unpack (expected 2, got 1)

### 패스워드 암호화 적용
* pip install bcrypt

In [145]:
import bcrypt